In [1]:
import os, sys
import reaxff_convert as rc
import pandas as pd
import numpy as np

from reaxFF_data import reaxFF_data, bidict

# Automated Parameter Collation

## Parameters Subsets
- Fe/O/Water: ffield.reax.Fe_O_C_H
- Water: ffield.reax.O_H
- Fe/C/H: ffield.reax.Fe_C_H
- CO2: ffield.reax.P_Na_Si_S_N_O_H_C

In [2]:
species = ['C', 'H', 'O', 'Fe']
potential_fd = os.path.join(os.getcwd(), "potentials")

water_fd = os.path.join(potential_fd, "ffield.reax.O_H")
fe_o_water_fd = os.path.join(potential_fd, "ffield.reax.Fe_O_C_H")
fe_c_h_fd = os.path.join(potential_fd, "ffield.reax.Fe_C_H")
co2_fd = os.path.join(potential_fd, "ffield.reax.P_Na_Si_S_N_O_H_C")

In [3]:
water_data = reaxFF_data(species)
water_data.read_lammps(water_fd)
water_data.clean_params()

fe_o_water_data = reaxFF_data(species)
fe_o_water_data.read_lammps(fe_o_water_fd)
fe_o_water_data.clean_params()

fe_c_h_data = reaxFF_data(species)
fe_c_h_data.read_lammps(fe_c_h_fd)
fe_c_h_data.clean_params()

co2_data = reaxFF_data(species)
co2_data.read_lammps(co2_fd)
co2_data.clean_params()

# Merging Parameters Together

This attempts to merge everything together, taking into account various new and improved fitting of later studies. Base case to be merged into would be *empty_data*.

In order, the sequence to be merged should be:

1. fe_o_water_data for the broadest coverage of useful activity
2. fe_c_h_data for the secondary coverage of important fe reactions
3. co2_data for the important carbonate reactions
4. water_data for the final good water interactions

In [4]:
master_data = reaxFF_data(species)
for i,k in master_data.params.items():
    k['source'] = float('nan')
steps = [master_data.copy()]

In [5]:
# First make the new df
steps.append(steps[-1].copy())

# # Combine data
for i, k in steps[-1].params.items():
    if i == 'general':
        k.update(fe_o_water_data.params[i])
        k['source'] = 'fe_o_water'
    else:
        cur_param_df = fe_o_water_data.params[i][k.columns.difference(['source'])]
        changes_df = k.reset_index().merge(cur_param_df.reset_index(),how="right").set_index('symbols')
        changes_df['source'] = 'fe_o_water'
        k.update(changes_df)

In [6]:
# First make the new df
steps.append(steps[-1].copy())

# # Combine data
for i, k in steps[-1].params.items():
    if i == 'general':
        continue
    else:
        cur_param_df = fe_c_h_data.params[i][k.columns.difference(['source'])]
        changes_df = k.reset_index().merge(cur_param_df.reset_index(),how="right").set_index('symbols')
        changes_df['source'] = 'fe_c_h'
        k.update(changes_df)

In [7]:
# First make the new df
steps.append(steps[-1].copy())

# # Combine data
for i, k in steps[-1].params.items():
    if i == 'general':
        continue
    else:
        cur_param_df = co2_data.params[i][k.columns.difference(['source'])]
        changes_df = k.reset_index().merge(cur_param_df.reset_index(),how="right").set_index('symbols')
        changes_df['source'] = 'co2'
        k.update(changes_df)

In [8]:
# First make the new df
steps.append(steps[-1].copy())

# # Combine data
for i, k in steps[-1].params.items():
    if i == 'general':
        continue
    else:
        cur_param_df = water_data.params[i][k.columns.difference(['source'])]
        changes_df = k.reset_index().merge(cur_param_df.reset_index(),how="right").set_index('symbols')
        changes_df['source'] = 'water'
        k.update(changes_df)

In [9]:
final_parameters = steps[-1].copy()
final_parameters.clean_params()
final_parameters

Description: None
Initializing Species: ['C', 'H', 'O', 'Fe']
**********************
general
**********************
        Overcoordination parameter 1 Overcoordination parameter 2  \
default                           50                       9.5469   

        Valency angle conjugation parameter 1  \
default                                1.6725   

        Triple bond stabilisation parameter 1  \
default                                1.7224   

        Triple bond stabilisation parameter 2 C2-correction  \
default                                6.8702        60.485   

        Undercoordination parameter 1 Triple bond stabilisation parameter 3  \
default                        1.0588                                   4.6   

        Undercoordination parameter 2 Undercoordination parameter 3  ...  \
default                       12.1176                       13.3056  ...   

        Valency angle conjugation parameter 2 Overcoordination parameter 3  \
default                       

# Converting back to LAMMPS Format

In [10]:
for i, step in enumerate(steps):
    step.clean_params()
    step.gen_species2id()
    
    new_FF = step.write_lammps()
    
    with open(os.path.join("test_potentials", "potential_step_{}.reax".format(i)), 'w') as f:
        f.write(new_FF)